# IMDB scraping

## Récupération des ID sur OMDB

Limité à 1000 par jour.

In [11]:
import omdb

In [2]:
omdb.set_default('apikey', "85a8320b") # identification

### Prise en main

In [3]:
lost = omdb.search('lost')

In [4]:
lost

[{'imdb_id': 'tt0082971',
  'poster': 'https://m.media-amazon.com/images/M/MV5BMjA0ODEzMTc1Nl5BMl5BanBnXkFtZTcwODM2MjAxNA@@._V1_SX300.jpg',
  'title': 'Raiders of the Lost Ark',
  'type': 'movie',
  'year': '1981'},
 {'imdb_id': 'tt0411008',
  'poster': 'https://m.media-amazon.com/images/M/MV5BNzhlY2E5NDUtYjJjYy00ODg3LWFkZWQtYTVmMzU4ZWZmOWJkXkEyXkFqcGdeQXVyNTA4NzY1MzY@._V1_SX300.jpg',
  'title': 'Lost',
  'type': 'series',
  'year': '2004–2010'},
 {'imdb_id': 'tt0335266',
  'poster': 'https://m.media-amazon.com/images/M/MV5BMTI2NDI5ODk4N15BMl5BanBnXkFtZTYwMTI3NTE3._V1_SX300.jpg',
  'title': 'Lost in Translation',
  'type': 'movie',
  'year': '2003'},
 {'imdb_id': 'tt0119567',
  'poster': 'https://m.media-amazon.com/images/M/MV5BMDFlMmM4Y2QtNDg1ZS00MWVlLTlmODgtZDdhYjY5YjdhN2M0XkEyXkFqcGdeQXVyNTI4MjkwNjA@._V1_SX300.jpg',
  'title': 'The Lost World: Jurassic Park',
  'type': 'movie',
  'year': '1997'},
 {'imdb_id': 'tt0104431',
  'poster': 'https://m.media-amazon.com/images/M/MV5BNDI1MzM0

Ok, donc omdb.search(titre) renvoie une liste avec tous les résultats qui matchent avec le titre. Heureusement, il y a le type qui nous aide. On veut donc récupérer tous les éléments de cette liste avec pour type 'series'. 

Pour *Lost*, on a qu'un seul élément avec le type série mais si on en a plusieurs, on prend que le premier ?

In [9]:
ncis = omdb.search('ncis')

In [15]:
ncis[0]['type']

'series'

As expected, on a plusieurs séries *NCIS*, la première étant l'originale. On devrait pouvoir avoir les deux autres en premier avec une recherche plus précise donc pas de soucis ici.

In [15]:
def get_IMDB_id_from_title(title) :
    """
    title : string, titre de la série dont on recherche les infos
    renvoie l'id IMDB associée à cette série s'il existe, -1 sinon
    """
    info = omdb.search(title)
    series = []
    for dico in info :
        if dico['type'] == 'series' :
            series.append(dico)
    if len(series) == 0 : # si jamais la recherche ne renvoie pas de séries
        #print("Pas de séries pour ce titre")
        return -1
    return series[0]['imdb_id']

In [23]:
get_IMDB_id_from_title('csi')

'tt0247082'

Ça marche ! https://www.imdb.com/title/tt0247082/reviews?sort=userRating&dir=desc&ratingFilter=0 est bien la page de la série CSI !

In [20]:
get_IMDB_id_from_title('fhjdsbghdjks')

Pas de séries pour ce titre


-1

Ça marche aussi quand la série n'existe pas.

### Récupération d'une partie des ID

Pour la recherche, on va utiliser les titres qu'on a un peu transformé pour Metacritic. Sachant que la limite est de 1000 par jour, on va voir aussi à quoi elle ressemble, cette limite.

In [8]:
titles = []
with open('titles.txt') as file :
    lignes = file.readlines()
    for ligne in lignes :
        l = ligne.split(' ')
        titles.append(l[0])

In [5]:
titles[0:10]

['battlestar-galactica-the-face-of-the-enemy',
 'sherlock',
 'drake-josh',
 'big-time-rush',
 'masterchef',
 'dive',
 'sabrina-the-teenage-witch',
 'the-ultimate-fighter',
 'my-boys',
 'aftermath-with-william-shatner']

### Day 1

On va faire un nouveau fichier avec 'titre imdb_id'. On n'écrit pas les séries qui n'ont pas d'ID pour aller plus vite par la suite.

In [31]:
series_rejetees = []
with open('titles_imdb_id.txt', 'w') as file2:
    for titre in titles :
        ID = get_IMDB_id_from_title(titre)
        if ID != -1 :
            file2.write(titre+" "+ID)
        else :
            series_rejetees.append(titre)

Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour ce titre
Pas de séries pour c

HTTPError: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?s=pramface&page=1&plot=short&apikey=85a8320b

In [32]:
len(series_rejetees)

215

Oups, qui c'est qui a oublié de mettre un '\n' ? On refait un petit compte hehe.

In [1]:
# https://github.com/kuk/log-progress
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [33]:
omdb.set_default('apikey', "dd6f2b4") # identification

In [36]:
cpt = 0
series_rejetees = []
with open('titles_imdb_id.txt', 'w') as file2:
    for titre in log_progress(titles, every=1) :
        ID = get_IMDB_id_from_title(titre)
        if ID != -1 :
            file2.write(titre+" "+ID+"\n")
        else :
            series_rejetees.append(titre)
        cpt += 1

VBox(children=(HTML(value=''), IntProgress(value=0, max=3514)))

HTTPError: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?s=dark-skies&page=1&plot=short&apikey=dd6f2b4

That's the limit ;)

In [37]:
len(series_rejetees)

217

On a 217 séries rejetées sur 1000 (pretty good).

In [39]:
series_rejetees[-1]

'miss-fisher-s-murder-mysteries'

In [6]:
titles[1000-1]

'miss-fisher-s-murder-mysteries'

### Day 2

En plus on peut faire avec les deux clés maintenant donc ça va aller plus vite.

In [7]:
cpt = 1000

In [8]:
titles_day2 = titles[cpt:]

In [12]:
series_rejetees = []

In [13]:
omdb.set_default('apikey', "dd6f2b4") # identification
with open('titles_imdb_id.txt', 'a') as file2: # 'a' pour ajouter à la suite du fichier
    for titre in log_progress(titles_day2, every=1) :
        ID = get_IMDB_id_from_title(titre)
        if ID != -1 :
            file2.write(titre+" "+ID+"\n")
        else :
            series_rejetees.append(titre)
        cpt += 1

VBox(children=(HTML(value=''), IntProgress(value=0, max=2514)))

HTTPError: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?s=grace-favour&page=1&plot=short&apikey=dd6f2b4

In [17]:
titles[1999]

'viva-la-bam'

In [18]:
titles_day2_2 = titles[2000:]

In [19]:
omdb.set_default('apikey', "85a8320b") # identification
with open('titles_imdb_id.txt', 'a') as file2: # 'a' pour ajouter à la suite du fichier
    for titre in log_progress(titles_day2_2, every=1) :
        ID = get_IMDB_id_from_title(titre)
        if ID != -1 :
            file2.write(titre+" "+ID+"\n")
        else :
            series_rejetees.append(titre)
        cpt += 1

VBox(children=(HTML(value=''), IntProgress(value=0, max=1514)))

HTTPError: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?s=the-hairy-bikers-northern-exposure&page=1&plot=short&apikey=85a8320b

In [21]:
series_rejetees[-1]

'bargain-fever-britain'

In [22]:
titles[2999]

'food-unwrapped'

### Day 3

In [9]:
titles_day3 = titles[2999:]

In [17]:
cpt = 0
omdb.set_default('apikey', "15d96de6") # identification
with open('titles_imdb_id.txt', 'a') as file2: # 'a' pour ajouter à la suite du fichier
    for titre in log_progress(titles_day3, every=1) :
        ID = get_IMDB_id_from_title(titre)
        if ID != -1 :
            file2.write(titre+" "+ID+"\n")

        cpt += 1

VBox(children=(HTML(value=''), IntProgress(value=0, max=515)))

## How to deal with the 'load more' button

Sur les pages IMDB, une partie des reviews est sous un bouton 'load more' donc il faut voir comment passer outre.

Selenium a l'air d'être la solution, à regarder plus précisément. 

### Selenium

In [64]:
#https://stackoverflow.com/questions/39112138/use-selenium-to-click-a-load-more-button-until-it-doesnt-exist-youtube

from selenium import webdriver      
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options

options = Options()
options.add_argument('--headless')

#driver = webdriver.Chrome(ChromeDriverManager().install())

#browser = webdriver.Firefox()#Chrome('./chromedriver.exe')

IMDB_HOME_PAGE_URL = "https://www.imdb.com/title/tt1338724/reviews?ref_=tt_ov_rt"


#driver = webdriver.Chrome('./chromedriver.exe')
#driver = webdriver.Firefox(FirefoxDriverManager().install())
driver = webdriver.Firefox(executable_path='/Users/constancescherer/Downloads/geckodriver', options=options)
driver.get(IMDB_HOME_PAGE_URL)



while True:
    try:
        #loadMoreButton = driver.find_element_by_xpath("//button[contains(@aria-label,'Load more')]")
        loadMoreButton = driver.find_element_by_id("load-more-trigger")
        #loadMoreButton = driver.find_element_by_xpath("//button")
        
        #element = driver.find_element_by_id("my-id")

        #actions = ActionChains(driver)
        #actions.move_to_element(loadMoreButton).perform()
        
        driver.execute_script("arguments[0].scrollIntoView();", loadMoreButton)
        
        time.sleep(2)
        loadMoreButton.click()
        time.sleep(2)
    except Exception as e:

        #print(e)
        break
#print("Complete")
html = driver.page_source
#time.sleep(2)
driver.quit()

In [2]:
print("ok")

ok


In [4]:
from bs4 import BeautifulSoup

In [65]:
html_soup = BeautifulSoup(html, 'html.parser')

In [82]:
reviews = html_soup.find_all('div', class_="lister-item-content")

In [88]:
len(reviews)

2

In [85]:
try :
    rating = int(reviews[0].find('div', class_="ipl-ratings-bar").span.span.text)
except Exception as e:
    print('fuck u')

fuck u


In [86]:
name = list(reviews[0].find('span', class_="display-name-link").children)[0].text
name

'bob the moo'

## 1 série

In [8]:
from selenium import webdriver      
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

In [13]:
def get_reviews_from_imdb(nom_serie, id_serie) :
    """
    nom_serie : string, nom de la série
    id_serie : string, id imdb de la série
    renvoie un dictionnaire {username : {serie : note}}
    """
    options = Options()
    options.add_argument('--headless')
    
    url = "https://www.imdb.com/title/"+id_serie+"/reviews?ref_=tt_ov_rt"
    
    # Tor needs to be running
    profile=webdriver.FirefoxProfile()
    profile.set_preference('network.proxy.type', 1)
    profile.set_preference('network.proxy.socks', '127.0.0.1')
    profile.set_preference('network.proxy.socks_port', 9050)

    driver = webdriver.Firefox(profile, executable_path='/Users/constancescherer/Downloads/geckodriver', options=options)
    driver.get(url)

    while True:
        try:
        
            loadMoreButton = driver.find_element_by_id("load-more-trigger")

            driver.execute_script("arguments[0].scrollIntoView();", loadMoreButton)
            
            time.sleep(2)
            loadMoreButton.click()
            time.sleep(2)
            
        except Exception as e:
            break

    html = driver.page_source
    
    driver.quit()
    
    html_soup = BeautifulSoup(html, 'html.parser')
    
    reviews = html_soup.find_all('div', class_="lister-item-content")
    nb_reviews = len(reviews)
    
    d = dict()
    for i in range(0, nb_reviews) :
        
        try : # on essaye de récupérer le rating
            rating = int(reviews[i].find('div', class_="ipl-ratings-bar").span.span.text)
        except Exception as e: # s'il existe pas on passe à la review suivante
            continue
        #rating = int(reviews[i].find('div', class_="ipl-ratings-bar").span.span.text)
        name = list(reviews[i].find('span', class_="display-name-link").children)[0].text
        
        d[name] = {nom_serie : rating}
        
    return d

In [12]:
d = get_reviews_from_imdb('sherlock', 'tt1475582')

552


Ok ça marche, mais ça a l'air de prendre du temps.

## 10 séries

In [14]:
from utils.progress_bar import *

In [15]:
titles = []
titles_id = dict()
with open('titles_imdb_id.txt', 'r') as file :
    lignes = file.readlines()
    for ligne in lignes :
        title, imdb_id = ligne.split(" ")
        imdb_id = imdb_id.rstrip('\n')
        
        titles.append(title)
        titles_id[title] = imdb_id

In [16]:
premiers = titles[0:10]

In [17]:
for nom_serie in log_progress(premiers, every=1) :
    imdb_id = titles_id[nom_serie]
    d = get_reviews_from_imdb(nom_serie, imdb_id)
    for username in d.keys() :
        if '/' in username : # on enlève les usernames avec un '/' dedans
            continue
        note = d[username][nom_serie]
        
        with open('ratings_imdb/users/'+username+'.txt', 'a') as file :
            file.write(nom_serie+" "+str(note)+"\n")
            
        with open('ratings_imdb/series/'+nom_serie+'.txt', 'a') as file:
            file.write(username+" "+str(note)+"\n")

VBox(children=(HTML(value=''), IntProgress(value=0, max=10)))